In [26]:
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
import requests
import pandas as pd


In [12]:

def get_champion_data(version, champ_name):
    url = f"https://ddragon.leagueoflegends.com/cdn/{version}/data/en_US/champion/{champ_name}.json"
    data = requests.get(url).json()["data"]
    for champ_name, champ_info in data.items():
        stats = {key: item for key, item in champ_info["stats"].items() if key not in ["crit", "critperlevel"]}
        info = {key: item for key, item in champ_info["info"].items() if key != "difficulty"}
        row = {
            **info,
            **stats
        }
    return list(row.values())
    


In [ ]:
champion_vector_mapping = {}
VERSION = "15.23.1"

def get_champion_vector(champ_name):
    if champ_name not in champion_vector_mapping:
        try:
            champion_vector_mapping[champ_name] = get_champion_data(VERSION, champ_name)
        except:
            champion_vector_mapping[champ_name] = [0] * 21
    return champion_vector_mapping[champ_name]

champ_cols = [
    "BLUE_TOP","BLUE_JUNGLE","BLUE_MIDDLE","BLUE_BOTTOM","BLUE_UTILITY",
    "RED_TOP","RED_JUNGLE","RED_MIDDLE","RED_BOTTOM","RED_UTILITY"
]
def row_to_vector(row):
    vectors = []
    for col in champ_cols:
        champ = row[col]
        vectors.extend(get_champion_vector(champ))
    return vectors



In [ ]:
df = pd.read_csv("preprocessed_matches.csv")
X = df.apply(row_to_vector, axis=1, result_type="expand")
y = df["RED_WINNER"].astype(int)

print(f"Feature matrix shape: {X.shape}")
print(f"Target vector shape: {y.shape}")

In [ ]:
scalar = StandardScaler()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [24]:
gb = GradientBoostingClassifier(random_state=42)

param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.05, 0.1, 0.2],
    'max_depth': [2, 3, 4],
    'subsample': [0.8, 1.0]
}

grid = GridSearchCV(
    gb,
    param_grid=param_grid,
    cv=3,
    scoring='accuracy',
    verbose=2,
    n_jobs=-1
)

grid.fit(X_train, y_train)

y_pred = grid.predict(X_test)

print("\nBest parameters:", grid.best_params_)
print("Best CV accuracy:", grid.best_score_)
print("\nTest accuracy:", accuracy_score(y_test, y_pred))
print("\nConfusion matrix:")
print(confusion_matrix(y_test, y_pred))

Fitting 3 folds for each of 36 candidates, totalling 108 fits

Best parameters: {'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 200, 'subsample': 1.0}
Best CV accuracy: 0.5358018207282913

Test accuracy: 0.5250262513125656

Confusion matrix:
[[985 494]
 [863 515]]
